<a href="https://colab.research.google.com/github/IntellSystemSEC/chatbot/blob/main/jornechatbot_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

import os
import gradio as gr
import numpy as np
import tensorflow as tf
import random
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')  # Ensure the punkt_tab resource is downloaded
from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import word_tokenize
import pickle

intents = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["hi", "hello", "hey"],
            "responses": ["Greetings! Ready to embark on a journey of travel planning?", "Hello there! Excited to assist with your travel plans!", "Hey! Planning a trip? Let's make it memorable!"]
        },
        {
            "tag": "farwel",
            "patterns": ["bye", "farewell", "see you"],
            "responses": ["Goodbye! Wishing you smooth travels!", "Farewell! Safe and joyful adventures await!", "See you later! May your journey be filled with incredible experiences!"]
        },
        {
            "tag": "destination_recommendation",
            "patterns": ["recommend a travel destination", "best places to travel", "where should I go"],
            "responses": ["Considering your interests, how about exploring the vibrant streets of Marrakech?", "How about a journey to the breathtaking landscapes of Patagonia?", "Ever thought about visiting the ancient wonders of Petra, Jordan?"]
        },
        {
            "tag": "budget_travel",
            "patterns": ["traveling on a budget", "affordable destinations", "budget-friendly trips"],
            "responses": ["Absolutely! Immerse yourself in the rich culture of Vietnam without breaking the bank.", "Budget travel is an adventure! Explore the charm of Portugal's coastal towns.", "No worries! Experience the magic of Colombia on a budget-friendly escapade."]
        },
        {
            "tag": "weather",
            "patterns": ["what's the weather like today", "tell me the weather forecast"],
            "responses": ["I'm sorry, I don't have access to real-time weather information. Check a reliable weather website or app for the latest updates."]
        },
        {
            "tag": "packing_tips",
            "patterns": ["packing tips", "what to pack for a trip", "travel essentials"],
            "responses": ["When packing, essentials like passport, charger, and versatile clothing are must-haves. Tailor your packing to the climate of your chosen destination."]
        },
        {
            "tag": "thanksgiving",
            "patterns": ["thanks", "thank you", "appreciate it"],
            "responses": ["You're welcome! If you have more travel questions, feel free to ask.", "No problem! Enjoy the excitement of planning your trip.", "Anytime! Safe and joyful travels!"]
        },
        {
            "tag": "destination_info",
            "patterns": ["tell me about Iceland", "information on Tokyo", "what to know about Barcelona"],
            "responses": ["Certainly! Iceland is a land of ice and fire, boasting glaciers, waterfalls, and the mesmerizing Northern Lights.", "In Tokyo, modernity meets tradition. Dive into a world of tech marvels, ancient temples, and bustling markets.", "Barcelona is a masterpiece of architecture with Gaudi's creations and a vibrant Mediterranean atmosphere."]
        },
        {
            "tag": "local_cuisine",
            "patterns": ["best local food in Bangkok", "try cuisine in Buenos Aires", "local dishes in Cape Town"],
            "responses": ["Indulge in the flavors of Bangkok with dishes like Pad Thai, Tom Yum, and Som Tum.", "Buenos Aires is a paradise for foodies! Savor empanadas, asado, and delectable Argentine steaks.", "Cape Town's culinary scene delights with Bobotie, Biltong, and fresh seafood by the coast."]
        },
        {
            "tag": "activity_recommendation",
            "patterns": ["recommend activities in Banff National Park", "things to do in Seoul", "activity suggestions for New Orleans"],
            "responses": ["In Banff National Park, hike to Lake Louise, explore Moraine Lake, and take a scenic drive on the Icefields Parkway.", "Seoul offers a mix of ancient and modern. Visit Gyeongbokgung Palace, shop in Myeongdong, and indulge in Korean BBQ.", "In New Orleans, immerse yourself in jazz on Frenchmen Street, savor beignets in the French Quarter, and take a steamboat cruise on the Mississippi River."]
        },
        {
            "tag": "currency_exchange",
            "patterns": ["currency exchange in Bali", "currency rate for Prague", "exchange money in Tokyo"],
            "responses": ["Exchange currency at reputable places in Bali. The current rate is approximately 14,000 Indonesian Rupiah for 1 US Dollar.", "When exchanging money in Prague, reliable offices offer around 22 Czech Koruna for 1 US Dollar.", "In Tokyo, find currency exchange services at major airports and banks. The rate is approximately 110 Japanese Yen for 1 US Dollar."]
        },
        {
            "tag": "travel_tips",
            "patterns": ["travel tips for Iceland", "advice for Patagonia trip", "tips when visiting the Scottish Highlands"],
            "responses": ["When in Iceland, dress in layers, explore the Golden Circle, and chase the enchanting Northern Lights.", "For a Patagonia trip, pack sturdy boots, explore Torres del Paine, and savor traditional Argentine barbecue.", "When visiting the Scottish Highlands, prepare for unpredictable weather, visit Loch Ness, and explore the historic Eilean Donan Castle."]
        },
        {
            "tag": "nature_spot",
            "patterns": ["recommend green spots", "best nature destinations", "where to find natural beauty"],
            "responses": ["Nature lovers, explore the wonders of Costa Rica's Monteverde Cloud Forest, home to diverse flora and fauna.", "Experience serenity at Plitvice Lakes in Croatia, where waterfalls cascade through lush greenery.", "Discover the beauty of Banff National Park in Canada, known for pristine landscapes and turquoise lakes."]
        },
        {
            "tag": "extra_category",
            "patterns": ["anything else", "extra category"],
            "responses": ["Feel free to ask me anything else! I'm here to provide information and recommendations on various travel topics.", "If you have more questions or need assistance with another category, feel free to let me know!"]
        },
        {
            "tag": "conversation_start",
            "patterns": ["start a conversation", "let's chat", "begin a discussion"],
            "responses": ["Hello! I'm here to help you with any travel-related questions or recommendations. What would you like to know?", "Hi there! Ready to talk about your travel plans? Ask me anything!"]
        },
        {
            "tag": "best_time_to_visit",
            "patterns": ["when is the best time to visit Bali", "ideal season for Santorini", "when to go to Yellowstone National Park"],
            "responses": ["The best time to visit Bali is during the dry season from April to October for sunny and pleasant weather.", "Santorini is popular in the summer (June to September) for warm weather and clear skies, but spring and fall are great with fewer crowds.", "The best time to go to Yellowstone National Park is from April to May and September to October for milder temperatures and fewer tourists."]
        },
        {
            "tag": "accommodation_recommendation",
            "patterns": ["recommend hotels in Paris", "best places to stay in Tokyo", "accommodation options in New York"],
            "responses": ["In Paris, consider luxurious hotels like Le Bristol, The Ritz, or Hotel de Crillon for an indulgent experience.", "Tokyo caters to various preferences. Explore Shinjuku, Shibuya, and Ginza for diverse accommodation options.", "New York offers diverse choices. Consider Times Square, Central Park, or SoHo for hotels suited to different budgets."]
        },
        {
            "tag": "solo_travel_destination",
            "patterns": ["solo travel destinations", "safest places for solo travel", "where to go alone"],
            "responses": ["For solo travelers, explore the safety and charm of Amsterdam, Barcelona, and Kyoto.", "Discover solo-friendly cities like Melbourne, Vancouver, and Copenhagen for a fulfilling solo travel experience.", "Enjoy the vibrant atmosphere of solo-friendly cities like Berlin, Dublin, and Edinburgh, known for rich culture and friendly locals."]
        },
        {
            "tag": "trip_planning",
            "patterns": ["can you suggest", "plan my trip", "make my trip", "I want to travel"],
            "responses": ["Absolutely! Ready to craft a personalized travel plan. Share your preferences, and let's make your trip extraordinary!"]
        },
        {
            "tag": "trip_duration",
            "patterns": ["long days trip", "short days trip", "extended vacation", "quick getaway"],
            "responses": ["Got it! Are you looking for an extended vacation to savor every moment, or a quick getaway for an exciting escape? Let me know your preferred duration."]
        },
        {
            "tag": "group_size",
            "patterns": ["traveling with 4 persons", "group trip with 8 people", "number of persons in my travel party"],
            "responses": ["Noted! Traveling with a group of 4 is fantastic. Any specific preferences or accommodations I should consider for your group?", "Exciting! Traveling with 8 people opens up possibilities. Let me know any preferences or special requests for your group.", "Perfect! Traveling with 4 persons requires special considerations. Feel free to share any specific requests or preferences."]
        },
        {
            "tag": "travel_preferences",
            "patterns": ["preferred travel scenarios", "what type of experiences do you like", "share your travel preferences"],
            "responses": ["Understanding your travel preferences is crucial. Are you into adventure, relaxation, cultural experiences, or a mix of everything?", "Fantastic! Let's tailor your trip based on your preferences. What type of experiences do you enjoy during your travels?", "Your travel preferences matter! Share your interests, and I'll suggest activities and destinations aligned with what you enjoy."]
        },
        {
            "tag": "travel_duration",
            "patterns": ["how many days should I spend", "ideal duration for a trip", "recommended travel duration"],
            "responses": ["Determining the ideal duration for your trip depends on your interests. Are you looking for a brief visit or an extended exploration? Let me know, and I'll provide personalized advice."]
        },
        {
            "tag": "local_guidance",
            "patterns": ["local tips", "advice from locals", "what locals recommend"],
            "responses": ["Local insights can elevate your trip! Any specific areas or aspects you'd like to explore based on local recommendations? Share your interests, and I'll provide guidance from locals."]
        },
        {
            "tag": "mention_destination",
            "patterns": ["I want to visit [destination]", "Tell me about [destination]", "Considering [destination] for my trip"],
            "responses": ["Great choice! [destination] is a fantastic destination. What specific information are you looking for?", "Exciting! [destination] has a lot to offer. What aspects of the trip do you need assistance with?", "Considering [destination]? Let's explore the best places and activities there!"]
        },
        {
            "tag": "custom_destination_info",
            "patterns": ["Tell me more about [destination]", "What's special about [destination]", "Give me insights on [destination]"],
            "responses": ["Absolutely! [destination] is known for its [unique features]. Whether it's [landmarks], [cuisine], or [local attractions], there's something for everyone."]
        },
        {
            "tag": "change_destination",
            "patterns": ["Change my destination to [new_destination]", "Considering [new_destination] instead", "How about [new_destination] for my trip"],
            "responses": ["Sure, let's explore [new_destination]! What interests you about this destination, and how can I assist you with your plans?", "Changing to [new_destination]? Fantastic choice! What specific details do you need for your trip to [new_destination]?"]
        },
        {
            "tag": "update_destination_preference",
            "patterns": ["Update my destination to [updated_destination]", "Switch to [updated_destination] instead", "I'm now interested in [updated_destination]"],
            "responses": ["Noted! Updating your destination to [updated_destination]. What preferences or information are you seeking for your trip to [updated_destination]?", "Great! [updated_destination] it is. Let me know your preferences, and I'll tailor the information accordingly."]
        },
        {
            "tag": "hotel_location",
            "patterns": ["hotel in"],
            "responses": ["Sure, let me find hotel locations for you! In which city are you looking for hotels?"]
        }
    ]
}
extended_intents = [
    {
        "tag": "plan_my_trip",
        "patterns": ["plan my trip", "suggest a destination", "help me with my travel plans"],
        "responses": ["Absolutely! I'd love to help plan your trip. Can you share your preferences?"]
    },
    {
        "tag": "general_greeting",
        "patterns": ["hi", "hello", "hey"],
        "responses": ["Hello! Ready for some travel talk?", "Hey there! Excited to chat about your next adventure!"]
    },
    {
        "tag": "farewell",
        "patterns": ["bye", "farewell", "see you"],
        "responses": ["Goodbye! Safe travels!", "Farewell! If you have more questions, feel free to ask."]
    },
    {
        "tag": "thank_you",
        "patterns": ["thanks", "thank you", "appreciate it"],
        "responses": ["You're welcome! Anything else I can assist you with?", "No problem! Enjoy your trip planning."]
    },
    {
        "tag": "get_local_tips",
        "patterns": ["local tips", "advice from locals", "what do locals recommend"],
        "responses": ["Local insights can make your trip special! Any specific area you're interested in?"]
    },
    {
        "tag": "weather_inquiry",
        "patterns": ["what's the weather like today", "tell me the weather forecast"],
        "responses": ["I don't have real-time weather info, but I can help with travel tips!"]
    },
    {
        "tag": "activity_recommendations",
        "patterns": ["things to do", "recommend activities", "what to see"],
        "responses": ["There's so much to explore! What kind of activities are you interested in?"]
    },
    {
        "tag": "change_destination",
        "patterns": ["change my destination to [new_destination]", "considering [new_destination] instead"],
        "responses": ["Sure, let's explore [new_destination]! What interests you about it?"]
    },
    {
        "tag": "update_preferences",
        "patterns": ["update my preferences", "change my travel preferences"],
        "responses": ["Of course! What experiences or activities are you into?"]
    },
    {
        "tag": "start_a_conversation",
        "patterns": ["start a conversation", "let's chat", "begin a discussion"],
        "responses": ["Hello! Ready to talk about your travel plans? Ask me anything!"]
    }
]

intents['intents'].extend(extended_intents)
# Initialize Lancaster Stemmer
stemmer = LancasterStemmer()

# Preprocessing: tokenize and organize intents
words = []
classes = []
documents = []
ignore_words = ['?']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Stem and lower-case words, remove duplicates
words = sorted(list(set([stemmer.stem(w.lower()) for w in words if w not in ignore_words])))
classes = sorted(list(set(classes)))

# Prepare training data
training = []
output = []
out_empty = [0] * len(classes)

train_x = []
train_y = []

for doc in documents:
    bag = []
    pattern_words = [stemmer.stem(word.lower()) for word in doc[0]]
    for w in words:
        bag.append(1 if w in pattern_words else 0)

    output_row = list(out_empty)
    output_row[classes.index(doc[1])] = 1

    train_x.append(bag)
    train_y.append(output_row)

# Shuffle and convert to NumPy arrays
combined = list(zip(train_x, train_y))
random.shuffle(combined)
train_x[:], train_y[:] = zip(*combined)

train_x = np.array(train_x)
train_y = np.array(train_y)

# Build the TensorFlow model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, input_shape=(len(train_x[0]),), activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(len(train_y[0]), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_x, train_y, epochs=200, batch_size=8, verbose=1)

# Save the model and preprocessing data
model.save('journebot_model.h5')
pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, open("preprocess_data.pkl", "wb"))

# Function to predict intents
def predict_intent(user_input, threshold=0.7):
    user_input_words = [stemmer.stem(word.lower()) for word in nltk.word_tokenize(user_input)]
    input_bag = [1 if word in user_input_words else 0 for word in words]
    model_input = np.array([input_bag])
    predictions = model.predict(model_input)[0]
    predicted_intent = classes[np.argmax(predictions)]
    confidence = predictions[np.argmax(predictions)]

    if confidence > threshold:
        return predicted_intent, confidence
    else:
        return "I'm not sure. Could you clarify?", None

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0163 - loss: 3.6388    
Epoch 2/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0247 - loss: 3.6344      
Epoch 3/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0441 - loss: 3.6198     
Epoch 4/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0138 - loss: 3.6240     
Epoch 5/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0604 - loss: 3.6050 
Epoch 6/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0679 - loss: 3.6015     
Epoch 7/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0458 - loss: 3.6019     
Epoch 8/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0758 - loss: 3.5845  
Epoch 9/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0593 - loss: 3.5909     
Epoch 10/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0844 - loss: 3.5689     
Epoch 11/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0844 - loss: 3.5631     
Epoch 12/200
14

In [ ]:
model = tf.keras.models.load_model('/content/journebot_model.h5')
data = pickle.load(open("preprocess_data.pkl", "rb"))
words = data['words']
classes = data['classes']

chat_history = []

def clean_up_sentence(sentence):
    sentence_words = word_tokenize(sentence)
    sentence_words = [LancasterStemmer().stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"Found in bag: {w}")
    return np.array(bag)

def chat(Jorne_user):
    p = bow(Jorne_user, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.2
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)

    # Debugging information
    print("Return List:", results)

    response = ""
    if results:
        intent = classes[results[0][0]]
        print("Intent:", intent)

        for i in intents['intents']:
            if i['tag'] == intent:
                response = random.choice(i['responses'])
                break

    # Add the current user input and response to chat history
    chat_history.append({'user_input': Jorne_user, 'bot_response': response})

    # Display chat history in the Gradio interface
    chat_history_display = "\n".join(
        f"User: {chat['user_input']}\nBot: {chat['bot_response']}" for chat in chat_history)

    return f"{response}\n\nChat History:\n{chat_history_display}"

# Gradio interface setup
iface = gr.Interface(fn=chat, inputs="text", outputs="text")
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aaba97ff64a271fb5a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
